### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [12]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [3]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'latt_spacing' : 0, # no order 1 term -- starts at 2
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'simultaneous', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [4]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
100% complete Time (s):  1.27100014687
Compiling results...


../fitter\special_functions.py:10: RuntimeWarning: invalid value encountered in double_scalars
  ysdev = np.abs(ss.kvp(n, gv.mean(g), 1)) * gv.sdev(g)


Done.

Total time (s):  4.64499998093 



In [113]:
print bootstrapper



Fit type: simultaneous (F^2 = FKFpi, bsN = 1)

Fitting to nlo 
 with lattice corrections O(0) 
 with volume corrections O(8) 
Fitted/[FLAG] values at physical point:

F_K / F_pi = 1.10785(72) [1.1932(19)]

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
  c_mpia2 0             -7e-12 +- 1            1              0.000 (10)
          1               0.0 (1.0)        L_5 0         -0.0005783 (91)
      L_4 0          0.0001587 (38)            1            0.00020 (10)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 4.3e+02 [28]    Q = 0    logGBF = -5752.8

Settings:
  svdcut/n = 1e-12/14    tol = (1e-08*,1e-10,1e-10)    (itns/time = 47/1.3)
  fitter = scipy_least_squares    method = trf



In [114]:
print bootstrapper.fits[0]

Least Square Fit:
  chi2/dof [dof] = 4.3e+02 [28]    Q = 0    logGBF = -5752.8

Parameters:
          mpi 0     0.097878 (67)       [ 0.097900 (67) ]  
              1      0.14074 (12)       [  0.14072 (12) ]  
              2      0.15833 (19)       [  0.15785 (19) ]  **
              3      0.18014 (13)       [  0.18116 (14) ]  *****
              4      0.08208 (25)       [  0.08126 (25) ]  ***
              5      0.13423 (17)       [  0.13428 (17) ]  
              6      0.13545 (31)       [  0.13557 (32) ]  
              7      0.18871 (17)       [  0.18870 (17) ]  
              8      0.21420 (20)       [  0.21397 (20) ]  *
              9      0.24220 (15)       [  0.24347 (16) ]  *****
             10      0.16518 (19)       [  0.16533 (19) ]  
             11      0.23583 (30)       [  0.23601 (30) ]  
             12      0.26500 (29)       [  0.26473 (29) ]  
             13      0.30103 (28)       [  0.30281 (30) ]  *****
           mk 0     0.228730 (99)       [ 0.228

### Or make all fits

In [13]:
p_dict = {
    'order' : {
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [14]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:
    for order_fit in ['nlo']: #['nlo', 'nnlo', 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Using default prior.
Making fits...
100% complete Time (s):  0.271999835968
Compiling results...
Done.
Saving...
Done.

Total time (s):  2.93600010872 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  0.284999847412
Compiling results...
Done.
Saving...
Done.

Total time (s):  2.95499992371 

ma nlo
Using default prior.
Making fits...
100% complete Time (s):  2.23399996758
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.32799983025 

ma-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  0.973999977112
Compiling results...
Done.
Saving...
Done.

Total time (s):  4.74000000954 



In [92]:
#bootstrapper.plot_parameter_histogram('L_5')

In [86]:
#print bootstrapper

In [87]:
#print bootstrapper.fits[0]

In [214]:
#bootstrapper.fits[0].show_plots()